# 1.Importação de Bibliotecas

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Preparação para Modelagem Preditiva
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Carregar e fazer leitura dos dados dentro do projeto
caminho_do_arquivo = "../desafio-ds-imdb-diel/desafio_indicium_imdb.csv"
data = pd.read_csv(caminho_do_arquivo)

ModuleNotFoundError: No module named 'seaborn'

# 2.Entendimento de problema

In [ ]:
# Verificar as linhas do data frame para inspeção
print('Visão dos dados:')
data = pd.DataFrame(data)
data.sample(7)

In [ ]:
print('As colunas são :\n')
print(data.columns)

In [ ]:
# Descrever os dados para analise e inspeção para limpeza
data.describe(include='all')

## 3.Análise Exploratória dos Dados (EDA)
### Limpeza e processamento dos Dados
* **Objetivo**: Demonstrar as principais características entre as variáveis e apresentar hipóteses relacionadas.
* **Ações**: Você deve incluir visualizações como gráficos de dispersão, histogramas, e boxplots para entender a distribuição dos dados e relações entre variáveis.
* **Ferramentas**: Utilize bibliotecas como `pandas`, `matplotlib`, e `seaborn` para esta parte.

In [ ]:
# Remover a coluna 'Unnamed : 0' que não é necessária para análise
data = data.drop(columns=["Unnamed: 0"])

# Converter 'Released_Year' para numérico, substituindo valores inválidos por NaN
data["Released_Year"] = pd.to_numeric(data["Released_Year"], errors="coerce")

# Remover caracteres não numéricos e virgulas da coluna 'Gross' e converter para numérico
data["Gross"] = data["Gross"].str.replace(",", "").astype(float)

# Substituir valores ausentes em 'Certificate' por 'Unknown'
data["Certificate"] = data["Certificate"].fillna("Unknown")

# Substituir valores ausentes em 'Meta_score' pela média dos valores disponíveis
meta_score_mean = data["Meta_score"].mean()
data["Meta_score"] = data["Meta_score"].fillna(meta_score_mean)

# Substituir valores ausentes em 'Gross' pela média dos valores disponíveis
gross_mean = data["Gross"].mean()
data["Gross"] = data["Gross"].fillna(gross_mean)

# Converter a coluna 'Runtime' para minutos numéricos
data["Runtime"] = data["Runtime"].str.replace(" min", "").astype(int)


In [ ]:
# Nemhum dados duplicados
data.duplicated().sum()

In [ ]:
# Verificar as limpezas
data.info()
data.head()

### Número de filmes por gêneros.

In [ ]:
# Configurar o estilo dos gráficos
sns.set_theme(style="whitegrid")

# Distribuição dos Gêneros
generos = data["Genre"].str.get_dummies(sep=", ")
contagem_generos = generos.sum().sort_values(ascending=False)

# Plotar a distribuição dos gêneros
plt.figure(figsize=(12, 8))
sns.barplot(
    x=contagem_generos.values,
    y=contagem_generos.index,
    hue=contagem_generos.index,
    palette="viridis",
    legend=False,
)
# Títutlo
plt.title("Distribuição dos Gêneros de Filmes")
plt.xlabel("Número de Filmes")
plt.ylabel("Gênero")
plt.show()

In [ ]:
print(f"{color.BOLD}Certificados mais Repetitidos")
print("*" * 30)
data["Certificate"].value_counts().to_frame().style.highlight_max(color="red")

* **Ação**: Analisar os filmes com as melhores notas no IMDB e maiores números de votos.
* **Ferramentas**: `pandas`para filtrar e ordenar os dados.

In [ ]:
# melhores filmes avaliados por votos à cada ano
data = data.sort_values(by="Released_Year", ascending=False)
melhores_filmes_avaliados = pd.DataFrame(columns=data.columns)

for year in data["Released_Year"].unique():
    if year not in [2025, 2024, 2023]:
        df_year = data[data["Released_Year"] == year]
        if not df_year.empty:
            df_year_sorted = df_year.sort_values(by="No_of_Votes", ascending=False)
            melhores_filmes_avaliado = df_year_sorted.iloc[0]
            melhores_filmes_avaliados = pd.concat(
                [melhores_filmes_avaliados, melhores_filmes_avaliado.to_frame().T],
                ignore_index=True,
            )

In [ ]:
# Mostrar os 10 melhores filmes avaliados por votos à cada ano
melhores_filmes_avaliados.head(10)

In [ ]:
# Análise Temporal de Bilheteria ao longo dos anos
from matplotlib.ticker import FuncFormatter

# Função para formatar os valores como dólares
def formatar_dolar(x, pos):
    return '${:.1f}M'.format(x*1e-6)

plt.figure(figsize=(12, 8))
sns.lineplot(data=data, x="Released_Year", y="Gross", marker="o")
plt.title("Evolução Aos longos dos anos de acordo com a Receita Bruta")
plt.xlabel("Ano de Lançamento")
plt.ylabel("Receita Bruta (em dólares)")

# Formatar o eixo y como dólares
formatter = FuncFormatter(formatar_dolar)
plt.gca().yaxis.set_major_formatter(formatter)
plt.show()

In [ ]:
# Obtenha os diretores com os maiores números de lançamentos
contagem_diretor = data["Director"].value_counts().sort_values(ascending=False)
contagem_diretor_df = contagem_diretor.to_frame().reset_index()
contagem_diretor_df.columns = ["Director", "count"]

# Plotar os dados
plt.figure(figsize=(12, 8))
sns.barplot(
    x="count",
    y="Director",
    hue="Director",
    data=contagem_diretor_df.head(10),
    palette="magma",
)
plt.title("Top 10 Directors com maiores lançamentos")
plt.xlabel("Número de filmes")
plt.ylabel("Diretores")
plt.show()

In [ ]:
# Análise de Diretores e Atores
# Top 10 Diretores por Avaliação IMDb
top_diretores = (
    data.groupby("Director")["IMDB_Rating"].mean().sort_values(ascending=False).head(10)
)

# Plotar os top 10 diretores
plt.figure(figsize=(12, 8))
sns.barplot(
    x=top_diretores.values,
    y=top_diretores.index,
    hue=top_diretores.index,
    palette="viridis",
)
plt.title("Top 10 Diretores por Avaliação IMDb")
plt.xlabel("Avaliação Média no IMDb")
plt.ylabel("Diretor")
plt.show()

# 4. Modelagem Preditiva
* Modelo Utilizado: `Regressão Linear` para prever a nota do IMDB.
* Tipo de Problema: Regressão.
* Avaliação do Modelo: `Mean Squared Error (MSE)` e `R-squared (R²)`.

In [ ]:

# Preencher valores ausentes com a moda em colunas categóricas
categorical_cols = [
    "Certificate",
    "Genre",
    "Director",
    "Star1",
    "Star2",
    "Star3",
    "Star4",
]
for col in categorical_cols:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].mode()[0])

# Codificação one-hot para colunas categóricas
data = pd.get_dummies(data, columns=categorical_cols)

# Seleção
X = data.drop(columns=["IMDB_Rating", "Series_Title", "Overview"])
y = data["IMDB_Rating"]

# Divisão dos dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Imputar valores ausentes nas colunas numéricas
imputer = SimpleImputer(strategy="mean")
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_teste = scaler.transform(X_test)

# Treinamento do modelo de Regressão Linear
model = LinearRegression()
model.fit(X_train, y_train)

# Previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliação do modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse}, R2: {r2}")

# 5.Previsão da Nota do IMDb para um Filme Específico
* **Ação**: Utilizar o modelo treinado para prever a nota do IMDB de um filme com as características fornecidas.
* **Ferramentas**: `pandas` para preparar os dados de entrada, `sklearn` para aplicar o modelo treinado.

In [ ]:
# Características do filme
filme = {
    "Series_Title": "The Shawshank Redemption",
    "Released_Year": 1994,
    "Certificate": "A",
    "Runtime": 142,
    "Genre": "Drama",
    "Meta_score": 80.0,
    "Director": "Frank Darabont",
    "Star1": "Tim Robbins",
    "Star2": "Morgan Freeman",
    "Star3": "Bob Gunton",
    "Star4": "William Sadler",
    "No_of_Votes": 2343110,
    "Gross": 28341469,
}

# Transformar os dados em um DataFrame
filme_df = pd.DataFrame([filme])

# Aplicar a mesma codificação one-hot
filme_df = pd.get_dummies(
    filme_df,
    columns=["Certificate", "Genre", "Director", "Star1", "Star2", "Star3", "Star4"],
)

# Alinhar o DataFrame do filme com as colunas do conjunto de treino
filme_df = filme_df.reindex(columns=X.columns, fill_value=0)

# Normalizar os dados do filme
X_film = scaler.transform(filme_df)

# Prever a nota do IMDb
nota_imdb = model.predict(X_film)
print(f"Nota do IMDb prevista: {nota_imdb[0]}")

In [ ]:
import joblib
# Salvar o modelo em formato .pkl
joblib.dump(model, 'modelo_imdb.pkl')